In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
from blackbox_wrapper import BlackboxWrapper
import keras
from datasets import build_gunpoint
from variational_autoencoder import load_model
from neighborhood_generators import NeighborhoodGenerator
from lore_wrapper import LoreWrapper
from utils import choose_z
from joblib import load
from lasts import load_multiple_lasts
import time
import numpy as np
from experiments_neighborhoods_surrogates import multiple_tests
from sbgdt import Sbgdt, generate_n_shapelets_per_size
from saxdt import Saxdt

Using TensorFlow backend.
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/francesco/anaconda3/envs/lasts_01/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

In [3]:
random_state = 0
np.random.seed(random_state)
dataset_name = "gunpoint"
blackbox_name = "knn"

file_path = parentdir + "/saved/" + dataset_name + "_" + blackbox_name + "_" + time.strftime("%Y%m%d_%H%M%S")

(X_train, y_train, X_val, y_val,
 X_test, y_test, X_exp_train, y_exp_train,
 X_exp_val, y_exp_val, X_exp_test, y_exp_test) = build_gunpoint(
    random_state = random_state
)

resnet = keras.models.load_model(parentdir + "/trained_models/gunpoint/gunpoint_resnet.h5")
knn = load(parentdir + "/trained_models/gunpoint/gunpoint_knn.joblib")

_, _, autoencoder = load_model(parentdir + "/trained_models/gunpoint/gunpoint_vae")

blackbox = BlackboxWrapper(resnet, 3, 2)
blackbox = BlackboxWrapper(knn, 2, 1)
encoder = autoencoder.layers[2]
decoder = autoencoder.layers[3]

DATASET INFO:
X SHAPE:  (50, 150, 1)
y SHAPE:  (50,)

CLASSES BALANCE
0 :  0.48
1 :  0.52

SHAPES:
BLACKBOX TRAINING SET:  (40, 150, 1)
BLACKBOX VALIDATION SET:  (10, 150, 1)
BLACKBOX TEST SET:  (150, 150, 1)
EXPLANATION TRAINING SET:  (40, 150, 1)
EXPLANATION VALIDATION SET:  (10, 150, 1)
EXPLANATION TEST SET:  (150, 150, 1)

Blackbox and Explanation sets are the same!
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
neighborhood_names = ["gaussian_matched", "uniform_sphere", "gaussian", "matched_uniform", "lore"]
neighborhood_list = [NeighborhoodGenerator, NeighborhoodGenerator, NeighborhoodGenerator, NeighborhoodGenerator, LoreWrapper]
neighborhood_kwargs_list = [
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian_matched",
            "sampling_kind": "gaussian_matched",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": False
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "uniform_sphere",
            "sampling_kind": "uniform_sphere",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": True
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian",
            "sampling_kind": "gaussian",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": False
        },
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian_matched",
            "sampling_kind": "uniform_sphere",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": True
        },
        {"verbose": False, "samples": 500, "ngen": 10}
    ]

In [5]:
n_shapelets_per_size = generate_n_shapelets_per_size(X_train.shape[1], n_shapelets_per_length=2, 
                                                     min_length=8, start_divider=2, divider_multiplier=1.26)
surrogate_kwargs_list = [{"shapelet_model_params": {"max_iter": 50, "n_shapelets_per_size":n_shapelets_per_size}, 
                          "random_state": random_state},
                             {"random_state": np.random.seed(0), "create_plotting_dictionaries":False}]

surrogate_list = [Sbgdt, Saxdt]
surrogate_names = ["stdt", "saxdt"]

In [6]:
%%time
out = multiple_tests(
        X_exp_test[::3],
        blackbox,
        encoder,
        decoder,
        neighborhood_list=neighborhood_list,
        neighborhood_kwargs_list=neighborhood_kwargs_list,
        neighborhood_names=neighborhood_names,
        surrogate_list=surrogate_list,
        surrogate_kwargs_list=surrogate_kwargs_list,
        surrogate_names=surrogate_names,
        file_path=file_path,
        random_state=random_state,
        n_instances_to_explain=1,
        verbose=True,
        simple_dump=[False, True],
        boxplots=True,
        labels=["Gun", "Point"]
)

TypeError: multiple_tests() missing 1 required positional argument: 'neighborhood_list'